<a href="https://colab.research.google.com/github/AliveBeing/Sarcasm-Detection-with-Convolutional-Neural-Network/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Load the dataset
df = pd.read_json('/content/sample_data/Sarcasm_Headlines_Dataset.json', lines=True)

# Check the data
print(df.head())


                                        article_link  \
0  https://www.huffingtonpost.com/entry/versace-b...   
1  https://www.huffingtonpost.com/entry/roseanne-...   
2  https://local.theonion.com/mom-starting-to-fea...   
3  https://politics.theonion.com/boehner-just-wan...   
4  https://www.huffingtonpost.com/entry/jk-rowlin...   

                                            headline  is_sarcastic  
0  former versace store clerk sues over secret 'b...             0  
1  the 'roseanne' revival catches up to our thorn...             0  
2  mom starting to fear son's web series closest ...             1  
3  boehner just wants wife to listen, not come up...             1  
4  j.k. rowling wishes snape happy birthday in th...             0  


In [ ]:
# Split the data into input (headlines) and output (labels)
sentences = df['headline'].values
labels = df['is_sarcastic'].values

# Tokenize the text data
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(sentences)

# Pad the sequences to ensure uniform input length
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


In [ ]:
vocab_size = len(word_index) + 1
embedding_dim = 16

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(24, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           474528    
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           10368     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                        

In [ ]:
epochs = 10
batch_size = 32

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))


Epoch 1/10
668/668 [==============================] - 13s 18ms/step - loss: 0.4778 - accuracy: 0.7649 - val_loss: 0.3501 - val_accuracy: 0.8463
Epoch 2/10
668/668 [==============================] - 12s 18ms/step - loss: 0.2353 - accuracy: 0.9163 - val_loss: 0.3487 - val_accuracy: 0.8527
Epoch 3/10
668/668 [==============================] - 12s 18ms/step - loss: 0.1066 - accuracy: 0.9674 - val_loss: 0.4473 - val_accuracy: 0.8398
Epoch 4/10
668/668 [==============================] - 12s 18ms/step - loss: 0.0419 - accuracy: 0.9902 - val_loss: 0.6424 - val_accuracy: 0.8383
Epoch 5/10
668/668 [==============================] - 11s 16ms/step - loss: 0.0166 - accuracy: 0.9959 - val_loss: 0.8271 - val_accuracy: 0.8390
Epoch 6/10
668/668 [==============================] - 12s 18ms/step - loss: 0.0094 - accuracy: 0.9981 - val_loss: 0.9687 - val_accuracy: 0.8375
Epoch 7/10
668/668 [==============================] - 12s 18ms/step - loss: 0.0071 - accuracy: 0.9985 - val_loss: 1.0984 - val_accuracy:

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.2f}')


167/167 [==============================] - 1s 4ms/step - loss: 1.3123 - accuracy: 0.8390
Test Accuracy: 0.84


In [ ]:
def predict_sarcasm(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(padded_sequence)
    return "Sarcastic" if prediction >= 0.5 else "Not Sarcastic"

sample_text = "The weather is absolutely wonderful today!"
print(predict_sarcasm(sample_text))


1/1 [==============================] - 0s 124ms/step
Not Sarcastic


In [ ]:
sample_text = "boehner just wants wife to listen, not come up with alternative debt-reduction ideas"
print(predict_sarcasm(sample_text))

1/1 [==============================] - 0s 21ms/step
Sarcastic


In [ ]:
sample_text = "I am busy right now, can I ignore you some other time?"
print(predict_sarcasm(sample_text))

1/1 [==============================] - 0s 22ms/step
Sarcastic
